In [1]:
import os
import pandas as pd
import load_geolife
import add_geohash
import create_nodes
import geohash_location_count
import geohash_user_count
import create_geohash_meet
import create_geohash_meet_time

In [2]:
def create_dirs():
    if not os.path.isdir('images/'):
        os.makedirs('images/')
    if not os.path.isdir('htmls/'):
        os.makedirs('htmls/')
    if not os.path.isdir('data/'):
        os.makedirs('data/')
    if not os.path.isdir('data2/'):
        os.makedirs('data2/')

In [3]:
def removeDuplicates():
    duplicates = pd.read_csv('data/geolife_duplicates_content.csv')
    df = pd.read_csv('data/geolife_geohash_size_8.csv')
    df['px'] = df['Person ID'].astype('string') + "_" + df['Trajectory'].astype('string') + ".plt"
    duplicates['px'] = duplicates['user'].astype('string')+"_"+duplicates['file_name'].astype('string')
    noduplicates = df[~df['px'].isin(duplicates['px'])].copy(deep=True)
    noduplicates.to_csv('data/geolife_geohash_size_8_no_duplicates.csv',index=False)


In [4]:
create_dirs()
load_geolife.load_geolife('./geolife/Data/')
add_geohash.add_geohash()

removeDuplicates()
create_nodes.create_nodes()
geohash_location_count.geohash_location_count()
geohash_user_count.geohash_user_count()
count_files = create_geohash_meet.createGeohashMeetJsons()
create_geohash_meet.createGeohashMeetCSV(count_files)

create_geohash_meet_time.create_geohash_time_meets()

  0%|          | 0/182 [00:00<?, ?it/s]

100%|██████████| 18165801/18165801 [08:47<00:00, 34445.00it/s]


loaded
415946
grouping...
grouped
1.0 415946 / 415946


100%|██████████| 414417/414417 [55:03<00:00, 125.46it/s] 


# Local

In [2]:
import pandas as pd
import json
from tqdm import tqdm

def create_geohash_time_meets_local():
	df = pd.read_csv('data/geolife_geohash_meet_intervals_size_8.csv')
	meets_dict = {}

	for index, row in tqdm(df.iterrows()):
		meets_intervals = json.loads(row['meets_intervals'])
		
		if len(meets_intervals) > 1:
			keys = list(meets_intervals.keys())
			intervals = [meets_intervals[key] for key in keys]
			
			for i in range(len(keys)):
				for j in range(i + 1, len(keys)): 
					for interval_i in intervals[i]:
						for interval_j in intervals[j]:
							if (interval_i[1] <= interval_j[2] and interval_j[1] <= interval_i[2]):
								if((keys[i] == '24' or keys[i] == '4') and (keys[j] == '24' or keys[j] == '4')):
									print(interval_i, interval_j)
								key = tuple(sorted([keys[i], keys[j]]))
								meets_dict.setdefault(row['geohash'], set()).add(key)

		else:
			key = next(iter(meets_intervals))
			meets_dict.setdefault(row['geohash'], set()).add(key)

	new_data = [{'geohash': geohash, 'person_ids': list(person_pairs)} for geohash, person_pairs in meets_dict.items()]
	new_df = pd.DataFrame(new_data)

	new_df.to_csv('data2/geolife_local_meets.csv', index=False)

create_geohash_time_meets_local()


177227it [00:24, 2207.76it/s] 

[4, '2009-03-01 14:02:57', '2009-05-11 08:23:03', 'wx4ex1g0', 7952590, 7952598] [24, '2009-03-25 23:54:45', '2009-03-25 23:54:45', 'wx4ex1g0', 9795864, 9795864]


415946it [00:40, 10146.00it/s]


# Global

In [1]:
import pandas as pd
import json
from tqdm import tqdm

def create_geohash_time_meets_global():
	df = pd.read_csv('data/geolife_geohash_meet_intervals_size_8.csv')
	meets_dict = {}

	for index, row in tqdm(df.iterrows()):
		meets_intervals = json.loads(row['meets_intervals'])
		
		if len(meets_intervals) > 1:
			keys = list(meets_intervals.keys())
			intervals = [meets_intervals[key] for key in keys]
			
			for i in range(len(keys)):
				for j in range(i + 1, len(keys)): 
					for interval_i in intervals[i]:
						for interval_j in intervals[j]:
							if (interval_i[1] <= interval_j[2] and interval_j[1] <= interval_i[2]):
								key = tuple(sorted([keys[i], keys[j]]))
								meets_dict.setdefault(key, {'users': key, 'meetings': [], 'geohashes': set(), 'time': [None, None]})
								meets_dict[key]['meetings'].append((row['Latitude'], row['Longitude']))
								meets_dict[key]['geohashes'].add(row['geohash'])
								if meets_dict[key]['time'][0] is None or interval_i[1] < meets_dict[key]['time'][0]:
									meets_dict[key]['time'][0] = interval_i[1]
								if meets_dict[key]['time'][1] is None or interval_j[2] > meets_dict[key]['time'][1]:
									meets_dict[key]['time'][1] = interval_j[2]

	new_data = [{'users': info['users'], 'meetings': info['meetings'], 'geohashes': list(info['geohashes']), 'time': info['time']} for info in meets_dict.values()]
	new_df = pd.DataFrame(new_data)

	new_df.to_csv('data2/geolife_global_meets.csv', index=False)

create_geohash_time_meets_global()


415946it [00:44, 9443.09it/s] 
